In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON, POST
import json, os 
from query_util_redundancy_filter import *
import sys
query_file   = 'query2-image_without_annotation_reproduce.sparql'
query_string = open(query_file,'r').read()
file_anno    = 'mmdetection/data/mixedDataset/annotations/instances_train2017_1.json'
dataset = process_query(query_string)
with open(file_anno,'w') as outfile:
    json.dump(dataset,outfile)
    
categories = [category['name'] for category in dataset['categories']]

number_of_categories = len(categories)

In [4]:
import numpy as np
import funcy
from sklearn.model_selection import train_test_split
import Ipynb_importer
from check_annotation import print_instances_class_histogram

def save_coco(file, images, annotations, categories):
    print_instances_class_histogram(file, annotations, [category['name'] for category in categories])
    with open(file, 'wt') as coco:
        json.dump({'images': images, 'annotations': annotations, 'categories': categories}, coco, indent=2,
                  sort_keys=False)

importing Jupyter notebook from check_annotation.ipynb


In [ ]:
def filter_annotations(annotations, images):
    image_ids = funcy.lmap(lambda i: int(i['id']), images)
    return funcy.lfilter(lambda a: int(a['image_id']) in image_ids, annotations)

In [ ]:
def dataset_split(annotation_file, path_to_train, path_to_test, ratio):
    with open(annotation_file, 'rt') as annotations:
        coco = json.load(annotations)
        images = coco['images']
        annotations = coco['annotations']
        categories = coco['categories']

        images_with_annotations = funcy.lmap(lambda a: int(a['image_id']), annotations)
        images = funcy.lremove(lambda i: i['id'] not in images_with_annotations, images)

        train, test = train_test_split(images, train_size=ratio)

        save_coco(path_to_train, train, filter_annotations(annotations, train), categories)
        save_coco(path_to_test, test, filter_annotations(annotations, test), categories)

In [ ]:
import json
path_to_train_val = 'mmdetection/data/mixedDataset/annotations/instances_train_val2017_person_pedestrian.json'
path_to_train = 'mmdetection/data/mixedDataset/annotations/instances_train2017_person_pedestrian.json'
path_to_val = 'mmdetection/data/mixedDataset/annotations/instances_val2017_person_pedestrian.json'
path_to_test = 'mmdetection/data/mixedDataset/annotations/instances_test2017_person_pedestrian.json'
ratio = 0.85
with open(path_to_val) as json_object:
    categories_dict =json.load(json_object)
categories= [category['name'] for category in categories_dict['categories']]
number_of_categories=len(categories)
dataset_split(file_anno, path_to_train_val, path_to_test, ratio)
dataset_split(path_to_train_val, path_to_train, path_to_val, ratio)

In [ ]:
!nvidia-smi

In [ ]:
# model.bbox_head.num_classes: classes number of extracted datasets

# data.samples_per_gpu: * gpu = batch size

# other parameters can also be set here and then mergerd into the file "***loadFromSeparatedFile.py"

In [ ]:
# new training pipeline on the coco-kitti mixedDataset with categories pedestrian and person_setting(person)

In [ ]:
%run mmdetection/tools/train.py \
mmdetection/configs/fcos/fcos_r50_caffe_fpn_gn-head_1x_mixedDataset_loadFromSeparatedFile.py \
--work-dir mmdetection/training_logs_example/fcos_coco_kitti_pedestrian_person_setting_bacth_4_lr_0.01/ \
--cfg-options model.bbox_head.num_classes={number_of_categories} \
data.train.ann_file={path_to_train} \
data.val.ann_file={path_to_val} \
runner.max_epochs=30 \
data.samples_per_gpu=4 \
load_from=mmdetection/pths/fcos/fcos_r50_caffe_fpn_gn-head_1x_coco.pth \
data.train.classes="$categories" \
data.val.classes="$categories" \
data.test.classes="$categories"

In [ ]:
%run mmdetection/tools/test.py \
mmdetection/configs/fcos/fcos_r50_caffe_fpn_gn-head_1x_mixedDataset_loadFromSeparatedFile.py \
mmdetection/training_logs_example/fcos_coco_kitti_pedestrian_person_setting_bacth_4_lr_0.01/latest.pth \
--work-dir mmdetection/training_logs_example/fcos_coco_kitti_pedestrian_person_setting_bacth_4_lr_0.01/ \
--out mmdetection/training_logs_example/fcos_coco_kitti_pedestrian_person_setting_bacth_4_lr_0.01/result_test_person.pkl \
--cfg-options data.test.ann_file={path_to_test} \
model.bbox_head.num_classes={number_of_categories} \
data.samples_per_gpu=4 \
data.train.classes="$categories" \
data.val.classes="$categories" \
data.test.classes="$categories" \
--eval bbox \
--show